In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data Feature Analysis**

## Exploratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data= pd.read_csv('/kaggle/input/malware-analysis-datasets-top1000-pe-imports/top_1000_pe_imports.csv')

In [ ]:
data.info()

In [ ]:
data.columns.tolist()

In [ ]:
pd.set_option('display.max_columns', None)
data.head(10)

In [ ]:
innocent= data.loc[data['malware']==0]
innocent.head(10)


In [ ]:
counts= data['malware'].value_counts()

print("Malicious Samples: ", counts[1])
print("Benign Samples: ", counts[0])

In [ ]:
%matplotlib inline

labels=['Malicious', 'Benign']
sizes=[data['malware'].value_counts()[1],
     data['malware'].value_counts()[0]]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels,)
ax1.axis('equal')
plt.show()

## Automatic Feature Selection

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [ ]:
y=data['malware']
x=data.drop(['malware','hash'],axis=1)
#dropping hash as it is a (mosty unique) character value, cannot be mapped into feature space.

#x.head()
#y.head()

In [ ]:
model=ExtraTreesClassifier(n_estimators=50, random_state=15372)
#Setting random state, else predictions vary everytime
model.fit(x,y)

In [ ]:
selections= SelectFromModel(model,prefit=True)
x_new=selections.transform(x)
print('Number of features: %d' %(x_new.shape[1]))

In [ ]:
nb_features= x_new.shape[1]
indices = np.argsort(model.feature_importances_)[::-1][:nb_features]
for f in range(nb_features):
    number = f+1
    feature_name = ''.join(x.columns[indices[f]])
    feature_importance = model.feature_importances_[indices[f]]
    print('   %d.\t%s \t%f' % (number, feature_name, (feature_importance * 100)))

In [ ]:
#Testing Purposes 
#print(indices)
#print(x.iloc[[1],[957]])

## Our Analysis: Top 1000 PE Headers

msvcrt.dll -

1. _cexit 8.280818
2. _controlfp 3.025105
3. _getmainargs 1.754543
4. _initterm 1.724222
5. _p__fmode 1.655268
6. exit 1.607496
7. _p__commode 1.430197
8. _set_app_type 1.413838
9. _exit 1.263665
10. _onexit 0.981152
11. _XcptFilter 0.709312
12. _setusermatherr 0.688940

Important Indicators while doing analysis:(my opinion based on my experience)

1. VirtualAlloc
2. VirtualProtect
3. GetModuleHandleA
4. ExitProcess
5. RegCloseKey
6. GetCurrentProcessId
7. TerminateProcess
8. LoadLibraryA
9. GetCurrentProcess
10. malloc
11. VirtualFree
12. free
13. MapVirtualKeyA
14. MapVirtualKeyW
15. SetWindowsHookExA
16. SetWindowsHookExW

There a couple of more interestung dll's that are used by malwares like WSOC which is used to communicate with the command and control center

## Feature Analysis: Malicious v. Benign Samples

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data= pd.read_csv('/kaggle/input/malware-analysis-datasets-top1000-pe-imports/top_1000_pe_imports.csv')

In [ ]:
data=data.drop(['hash'],axis=1)
data.info() #dropping hash; to get all integer values

In [ ]:
flag=0
for column in data.columns.tolist():
    if data[column].nunique()!=2:
        print(column+ " has more than 2 unique values")
        flag=1

if flag==0:
    print("All columns have 2 values only")

In [ ]:
mal= data.loc[data['malware']==1]
benign= data.loc[data['malware']==0]

In [ ]:
index= data.columns.tolist()
columns=['Benign=0', 'Benign=1', 'Malicious=0', 'Malicious=1', 'Difference']

In [ ]:
df = pd.DataFrame(index=index, columns=columns)
df = df.fillna(0)
#df.head()

In [ ]:
#Creating a dataframe with the percentage of benign and malicious samples corresponding to 0 and 1 respectively for each column

for column in data.columns.tolist():
    df.loc[[column],['Benign=1']]= np.round((benign[column].sum()/benign.shape[0]*100),2)
    df.loc[[column],['Benign=0']]= np.round(((benign.shape[0]-benign[column].sum())/benign.shape[0]*100),2)
    df.loc[[column],['Malicious=1']]= np.round((mal[column].sum()/mal.shape[0]*100),2)
    df.loc[[column],['Malicious=0']]= np.round(((mal.shape[0]-mal[column].sum())/mal.shape[0]*100),2)
    df.loc[[column],['Difference']]= abs(np.round((((benign[column].sum()/benign.shape[0])- (mal[column].sum()/mal.shape[0]))*100 ),2))

In [ ]:
df.head()

In [ ]:
# Sorting with respect to differences, to get the most different values

df=df.sort_values('Difference', ascending=False)
df=df.drop(['malware']) #Obviously, this column will be different
#df.head(20)

# Model Comparision

Comparing the performance of our model with all features of the dataset, and important features selected automatically in sk-learn.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
data= pd.read_csv('/kaggle/input/malware-analysis-datasets-top1000-pe-imports/top_1000_pe_imports.csv')

In [ ]:
def create_data(dataframe):
    y= dataframe['malware']
    X= dataframe.drop(['malware'], axis=1)
    print("X Shape: ", X.shape)
    print("Y Shape: ", y.shape)
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=2)
    
    return (X, y, X_train, X_test, y_train, y_test)

In [ ]:
def evaluation(model, X_val, y_val, predictions):
    print("Confusion Matrix: \n", confusion_matrix(y_val, predictions))
    print("Precision:", precision_score(y_val, predictions))
    print("Recall:",recall_score(y_val, predictions))
    print("F1 Score:", f1_score(y_val, predictions))
    return

## Automatically Selected Important Features

In [ ]:
df= data

In [ ]:
y= df['malware']
X= df.drop(['hash','malware'], axis=1)
#Remove hash, it is always a unique character string- MODEL WILL NOT TRAIN

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
rf = RandomForestClassifier(n_estimators=50, random_state=3)
rf.fit(X_train,y_train)

In [ ]:
#Find Important features
importances= pd.DataFrame({'Feature': X_train.columns, 'Importance': np.round((rf.feature_importances_ * 100),3)})
importances= importances.sort_values('Importance', ascending=False).set_index('Feature')

importances.loc[importances['Importance']> 0.2]

In [ ]:
features= importances.loc[importances['Importance']> 0.2].index.to_list()
mal= ['malware']
columns= features + mal

columns

In [ ]:
#Keep only important features

df=data[columns]

#df.info()

In [ ]:
X, y, X_train, X_test, y_train, y_test= create_data(df)


In [ ]:
rf = RandomForestClassifier(n_estimators=100, 
                            random_state=3, 
                            bootstrap= True,
                            max_depth= 110,
                            max_features= 3,
                            min_samples_leaf= 2,
                            min_samples_split= 3)
rf.fit(X_train,y_train)

In [ ]:
y_pred= rf.predict(X_test)

print("Train Accuracy:",rf.score(X_train, y_train) )#accuracy
print( "Test Accuracy:", accuracy_score(y_test, y_pred, normalize=True))

In [ ]:
scores= cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy') #5 fold

print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())

In [ ]:
#Cross Validation Evaluation

print("Cross Validation Evaluation:\n")
predictions = cross_val_predict(rf, X_train, y_train, cv=3)
evaluation(rf, X_train, y_train, predictions)

In [ ]:
#Test Set Evaulation

print("\nTest Set Evaluation:\n")
y_pred = rf.predict(X_test)
evaluation(rf, X_test, y_test, y_pred)

# Testing and Tuning the Model
Using our selected features

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data= pd.read_csv('/kaggle/input/malware-analysis-datasets-top1000-pe-imports/top_1000_pe_imports.csv')

In [ ]:
df= data[['OpenProcess', 'GetCurrentProcess', 'GetProcessHeap','ReadFile', 'CreateFileW', 'WriteFile', 'FindFirstFileW', 'FindNextFileW', 'SetWindowsHookExW', 'GetAsyncKeyState', 'GetForegroundWindow', 'GetKeyState', 'MapVirtualKeyW', 'VirtualAlloc', 'VirtualProtect', 'GetModuleHandleA', 'ExitProcess', 'RegCloseKey','GetCurrentProcessId', 'malware']]
#df.columns.to_list()


In [ ]:
y= df['malware']
X= df.drop(['malware'], axis=1)
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=2)
#print(X_train.shape, X_test.shape)
#print(y_train.shape, y_test.shape)

In [ ]:
#Random Forest Classifier

rf = RandomForestClassifier(n_estimators=50, random_state=3)
rf.fit(X_train,y_train)


## Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
y_pred= rf.predict(X_test)

print("Train Accuracy:",rf.score(X_train, y_train) )
print( "Test Accuracy:", accuracy_score(y_test, y_pred, normalize=True))

In [ ]:
#since accuracy is pretty high (97.8%), will check on cross validation

scores= cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy') #5 fold

print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())

In [ ]:
#Still pretty good accuracy, so checking Out of Bag score

rf = RandomForestClassifier(n_estimators=50, oob_score = True, random_state=3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("OOB score:", round(rf.oob_score_, 4)*100, "%")

In [ ]:
def evaluation(model, X_val, y_val, predictions):
    print("Confusion Matrix: \n", confusion_matrix(y_val, predictions))
    print("Precision:", precision_score(y_val, predictions))
    print("Recall:",recall_score(y_val, predictions))
    print("F1 Score:", f1_score(y_val, predictions))
    return

In [ ]:
#Cross Val Evaluation

print("Cross Validation:\n")
predictions = cross_val_predict(rf, X_train, y_train, cv=3)
evaluation(rf, X_train, y_train, predictions)


#Confusion matrix: [[TN, FP], [FN,TP]]

In [ ]:
#Testing Evaluation

print("Test Set:\n")
y_pred = rf.predict(X_test)
evaluation(rf, X_test, y_test, y_pred)

High Precision= Low false positive rate
High Recall= Low false negative rate

So I am seeing some false positives. But good news is that we have VVV high true positives and VVV low false negatives, which is required for a Malware Detector

## Feature Importance

In [ ]:
#Checking what Features are deamed important by the classifier; cross check with how it relates to Bhat's Stuff

importances= pd.DataFrame({'Feature': X_train.columns, 'Importance': np.round(rf.feature_importances_,3)})
importances= importances.sort_values('Importance', ascending=False).set_index('Feature')

importances


## Hyperparameter Testing

In [ ]:
# Current parameters in use
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters I want to work on:

1. bootstrap = method for sampling data points (with or without replacement) (False= entire dataset used to make trees)
2. max_depth = max number of levels in each decision tree (More depth may lead to single leaf nodes in each tree, less depth may not be sufficient enough to capture the info)
3. max_features = max number of features considered for splitting a node
4. n_estimators = number of trees in the forest
5. min_samples_split = min number of data points placed in a node before the node is split (how many samples are needed to split the node)
6. min_samples_leaf = min number of data points allowed in a leaf node (helps smoothen the model)

In [ ]:
#Testing diff values: each iteration, algo chooses a different combination of features

from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
rf = RandomForestClassifier() #Base Model
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42,
                               n_jobs = -1)

# 3 folds of cross validation, more cv reduces overfitting but increases run time.
# n_jobs=-1 uses all processors in parallel
# Verbosity helps log the output. https://stats.stackexchange.com/questions/153823/what-is-verbose-in-scikit-learn-package-of-python

In [ ]:
#Will take a while to run
rf_random.fit(X_train, y_train)

In [ ]:
#Show best parameters
rf_random.best_params_


Found Best Parameters:

1. n_estimators: 1600
2. min_samples_split: 10
3. min_samples_leaf: 2
4. max_features: sqrt
5. max_depth: 80
6. bootstrap: True

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [8,10],
    'n_estimators': [1400, 1600, 1800]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

Best Parameters:

1. 'bootstrap': True,
2. 'max_depth': 110,
3. 'max_features': 3,
4. 'min_samples_leaf': 2,
5. 'min_samples_split': 10,
6. 'n_estimators': 1600

# Malware Detector (Final)

(After Tuning)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
data= pd.read_csv('/kaggle/input/malware-analysis-datasets-top1000-pe-imports/top_1000_pe_imports.csv')

In [ ]:
df= data[['OpenProcess', 'LoadLibraryA', 'GetProcessHeap', 'ShellExecuteW', 'VirtualFree', 'GetDC', 'IsDebuggerPresent', 'malloc', 'FindNextFileA', 'free', 'GetAsyncKeyState', 'GetTickCount', 'exit', '_cexit', 'VirtualAlloc', 'VirtualProtect', 'GetModuleHandleA', 'ExitProcess', 'RegCloseKey','GetCurrentProcessId', 'malware']]

In [ ]:
y= df['malware']
X= df.drop(['malware'], axis=1)
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=2)
#print(X_train.shape, X_test.shape)
#print(y_train.shape, y_test.shape)

In [ ]:
rf = RandomForestClassifier(n_estimators=1600, 
                            random_state=3, 
                            bootstrap= True,
                            max_depth= 110,
                            max_features= 3,
                            min_samples_leaf= 2,
                            min_samples_split= 10)
rf.fit(X_train,y_train)

#If Bootstrap is False then OOB is not available

## Model Evaluation

In [ ]:
y_pred= rf.predict(X_test)

print("Train Accuracy:",rf.score(X_train, y_train) )#accuracy
print( "Test Accuracy:", accuracy_score(y_test, y_pred, normalize=True))

In [ ]:
scores= cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy') #5 fold

print("Scores: ", scores)
print("Mean: ", scores.mean())
print("Standard Deviation: ", scores.std())

In [ ]:
def evaluation(model, X_val, y_val, predictions):
    print("Confusion Matrix: \n", confusion_matrix(y_val, predictions))
    print("Precision:", precision_score(y_val, predictions))
    print("Recall:",recall_score(y_val, predictions))
    print("F1 Score:", f1_score(y_val, predictions))
    return

In [ ]:
#Cross Validation Evaluation

print("Cross Validation Evaluation:\n")
predictions = cross_val_predict(rf, X_train, y_train, cv=3)
evaluation(rf, X_train, y_train, predictions)

In [ ]:
#Test Set Evaulation

print("\nTest Set Evaluation:\n")
y_pred = rf.predict(X_test)
evaluation(rf, X_test, y_test, y_pred)